# This is our testing notebook!

### The concept here is to have a reusable series of tests, for testing the integration of our ML models into the application. 

### Verify that Nvidia drivers, Cuda, and Pytorch are functional and GPU ready! (if this doesnt pass then fix your system before working with the Neural Network model, everything else should work fine)

In [1]:
import torch
import subprocess

def is_cuda_installed():
    """
    Check if `nvidia-smi` is installed and available.

    Returns:
        Tuple: (bool, str)
            - True if `nvidia-smi` is available, False otherwise.
            - Output of `nvidia-smi` if available, otherwise an error message.
    """
    try:
        result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            return True, result.stdout
        return False, result.stderr
    except FileNotFoundError:
        return False, "nvidia-smi not found. Ensure the NVIDIA drivers are properly installed."

# Check GPU, CUDA, and PyTorch setup
def run_gpu_test():
    """
    Checks for an available GPU, prints useful info to stdout, and runs a simple test.

    Returns:
        bool: True if GPU is available and test passes, False otherwise.
    """
    # Check if CUDA is available
    if not torch.cuda.is_available():
        print("GPU is not available")
        return False

    print("\nGPU is available")
    print("CUDA version:", torch.version.cuda)
    print("PyTorch CUDA version:", torch.version.cuda)
    print("Number of GPUs:", torch.cuda.device_count())
    print("Device name:", torch.cuda.get_device_name(0))

    # Check `nvidia-smi`
    installed, output = is_cuda_installed()
    if not installed:
        print("\n`nvidia-smi` is not installed or not found.\n"
              "Please see https://docs.nvidia.com/cuda/cuda-installation-guide-linux/")
        return False
    print("\n`nvidia-smi` is available:\n")
    print(output)

    # Run a simple PyTorch tensor operation on GPU
    print("Running a simple tensor operation on the GPU...")
    try:
        x = torch.rand(10000, 10000, device='cuda')
        y = torch.mm(x, x)
        print("GPU test passed successfully!")
        return True
    except Exception as e:
        print(f"GPU test failed: {e}")
        return False

# Run the test
run_gpu_test()



GPU is available
CUDA version: 11.8
PyTorch CUDA version: 11.8
Number of GPUs: 1
Device name: NVIDIA GeForce RTX 3070

`nvidia-smi` is available:

Sun Nov 24 13:24:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070        Off |   00000000:01:00.0  On |                  N/A |
|  0%   53C    P8             19W /  220W |    2213MiB /   8192MiB |      3%      De

True

### This block tests inference.py script and trained models for functionality, before attempting to tie it into the front end.

In [2]:
import sys
sys.path.append('../python')  # Ensure the python folder is in the module path
from inference import predict_election_from_state_with_metrics
import json

if __name__ == "__main__":
    # Add all models to the list of models to test
    test_models = [
        "logistic_regression",
        "k_nearest_neighbors",
        "decision_tree",
        "random_forest",
        "neural_network",
    ]

    test_cases = [
        {'state': 'California', 'year': 2024},
        {'state': 'Texas', 'year': 2024},
        {'state': 'Nevada', 'year': 2024},
        {'state': 'Florida', 'year': 2024},
    ]

    for model_name in test_models:
        print(f"\n=== Testing model: {model_name} ===")
        for case in test_cases:
            try:
                state = case['state']
                year = case['year']
                print(f"\nPredicting for {state} ({year}) using {model_name}...")
                
                # Call the function and unpack the result
                result = predict_election_from_state_with_metrics(model_name, state, year)
                prediction = result.get('predicted_result')
                metrics = result.get('metrics', {})
                accuracy = metrics.get('accuracy', 'N/A')
                classification_report = metrics.get('classification_report', {})

                # Display prediction
                print(f"  Predicted Election Result: {prediction}")
                print(f"  Model Accuracy: {accuracy:.2f}")

                # Display classification report
                print("  Classification Report:")
                for label, values in classification_report.items():
                    if isinstance(values, dict):  # Skip aggregated metrics like 'accuracy'
                        print(f"    Label {label}:")
                        print(f"      Precision: {values.get('precision', 'N/A'):.2f}")
                        print(f"      Recall: {values.get('recall', 'N/A'):.2f}")
                        print(f"      F1-Score: {values.get('f1-score', 'N/A'):.2f}")
                        print(f"      Support: {values.get('support', 'N/A')}")
            except Exception as e:
                print(f"Error: {e}")



=== Testing model: logistic_regression ===

Predicting for California (2024) using logistic_regression...
  Predicted Election Result: Democratic
  Model Accuracy: 0.85
  Classification Report:
    Label 0:
      Precision: 0.82
      Recall: 0.94
      F1-Score: 0.87
      Support: 33.0
    Label 1:
      Precision: 0.91
      Recall: 0.74
      F1-Score: 0.82
      Support: 27.0
    Label macro avg:
      Precision: 0.86
      Recall: 0.84
      F1-Score: 0.84
      Support: 60.0
    Label weighted avg:
      Precision: 0.86
      Recall: 0.85
      F1-Score: 0.85
      Support: 60.0

Predicting for Texas (2024) using logistic_regression...
  Predicted Election Result: Republican
  Model Accuracy: 0.85
  Classification Report:
    Label 0:
      Precision: 0.82
      Recall: 0.94
      F1-Score: 0.87
      Support: 33.0
    Label 1:
      Precision: 0.91
      Recall: 0.74
      F1-Score: 0.82
      Support: 27.0
    Label macro avg:
      Precision: 0.86
      Recall: 0.84
      F1

### testing of all states logic

In [3]:
import sys
sys.path.append('../python')  # Ensure the python folder is in the module path
from inference import predict_election_for_all_states

if __name__ == "__main__":
    # Define the models and year to test
    test_models = [
        "logistic_regression",
        "k_nearest_neighbors",
        "decision_tree",
        "random_forest",
        "neural_network",
    ]
    test_year = 2024

    for model_name in test_models:
        print(f"\n=== Testing all states with model: {model_name} ===")
        try:
            # Predict for all states
            predictions = predict_election_for_all_states(model_name, year=test_year)
            
            # Display predictions in a human-readable format
            for state, result in predictions.items():
                if "error" in result:
                    print(f"  State: {state} - Error: {result['error']}")
                else:
                    print(f"  State: {state} - Predicted Result: {result['predicted_result']}")

        except Exception as e:
            print(f"Error testing model {model_name}: {e}")



=== Testing all states with model: logistic_regression ===
  State: Alabama - Predicted Result: Republican
  State: Alaska - Predicted Result: Republican
  State: Arizona - Predicted Result: Republican
  State: Arkansas - Predicted Result: Republican
  State: California - Predicted Result: Democratic
  State: Colorado - Predicted Result: Democratic
  State: Connecticut - Predicted Result: Democratic
  State: Delaware - Predicted Result: Democratic
  State: Florida - Predicted Result: Republican
  State: Georgia - Predicted Result: Republican
  State: Hawaii - Predicted Result: Democratic
  State: Idaho - Predicted Result: Republican
  State: Illinois - Predicted Result: Democratic
  State: Indiana - Predicted Result: Republican
  State: Iowa - Predicted Result: Republican
  State: Kansas - Predicted Result: Republican
  State: Kentucky - Predicted Result: Republican
  State: Louisiana - Predicted Result: Republican
  State: Maine - Predicted Result: Democratic
  State: Maryland - Pred